In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import cv2
import mediapipe as mp
import os
import subprocess
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Prepare Data Testing Image 

In [2]:
def list_jpg_png_files(directory):
    jpg_png_files = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(root, filename)
                file_path = file_path.replace('\\', '/')
                jpg_png_files.append(file_path)
    return jpg_png_files

list_jpg_png_files('./datasets/Testing-Image')

['./datasets/Testing-Image/11_N_92.jpg',
 './datasets/Testing-Image/11_q_.jpg',
 './datasets/Testing-Image/1_ibu_.jpg',
 './datasets/Testing-Image/87_kerja_1.jpg',
 './datasets/Testing-Image/8_kerja_1.jpg',
 './datasets/Testing-Image/8_malam_1.jpg',
 './datasets/Testing-Image/8_malam_2.jpg',
 './datasets/Testing-Image/8_siang_2.jpg',
 './datasets/Testing-Image/91_jawab_1.jpg',
 './datasets/Testing-Image/91_kantor_1.jpg',
 './datasets/Testing-Image/92_jawab_1.jpg',
 './datasets/Testing-Image/92_kantor_1.jpg',
 './datasets/Testing-Image/93_jawab_1.jpg',
 './datasets/Testing-Image/93_sore_1.jpg',
 './datasets/Testing-Image/95_bagaimana_1.jpg',
 './datasets/Testing-Image/95_bagaimana_2.jpg',
 './datasets/Testing-Image/96_bagaimana_1.jpg',
 './datasets/Testing-Image/96_besok_1.jpg',
 './datasets/Testing-Image/96_kemarin_2.jpg',
 './datasets/Testing-Image/96_nanti_1.jpg',
 './datasets/Testing-Image/97_lusa_1.jpg',
 './datasets/Testing-Image/97_nanti_1.jpg',
 './datasets/Testing-Image/98_seka

# Test Static Image

In [3]:
def extract_hand_features(image_path):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5
    )

    data = []

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.append(landmark.x)
                landmarks.append(landmark.y)
                landmarks.append(landmark.z)
            data.append([os.path.basename(image_path)] + landmarks)

    hands.close()

    return data


# sample_data = extract_hand_features("datasets/Testing-Image/0_Apa_1.jpg")[0][1:]
samples = []
samples_path = []
for file in list_jpg_png_files('./datasets/Testing-Image'):
    samples_path.append(file)
    samples += extract_hand_features(file)

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [4]:
samples_path[:5]

['./datasets/Testing-Image/11_N_92.jpg',
 './datasets/Testing-Image/11_q_.jpg',
 './datasets/Testing-Image/1_ibu_.jpg',
 './datasets/Testing-Image/87_kerja_1.jpg',
 './datasets/Testing-Image/8_kerja_1.jpg']


# Test Predict

In [5]:
import joblib
import numpy as np

scaler = joblib.load("scaler.pkl")

for i in range(len(samples)):
    # # with scaler 
    samples[i] = samples[i][1:]
    samples[i] = np.array(samples[i]).reshape(1, -1)
    # samples[i] = scaler.transform(samples[i])

    # without scaler
    # samples[i] =  np.expand_dims(samples[i][1:], axis=0)

label_encoder = joblib.load("label_encoder.pkl")

In [6]:
import tensorflow as tf 

model_name = "bundling-scale-and-calssification-19am"
model_path = f"{model_name}.h5"
# Memuat kembali model tanpa custom_objects
model_load = tf.keras.models.load_model(model_path)
print(model_load.summary())



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 63)]              0         
                                                                 
 sequential_28 (Sequential)  (None, 66)                8610      
                                                                 
 dense (Dense)               (None, 64)                4288      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 66)                2178      
                                                                 
Total params: 17156 (67.02 KB)
Trainable params: 17028 (66.5

In [7]:
# import tensorflow as tf

# # model_path = 'models/14-juni-1058am.h5'
# model_path = './models/bundling-scaler-classification.h5'
# model = tf.keras.models.load_model(model_path)
# # model.summary()

In [8]:
import pandas as pd
from difflib import get_close_matches

class_labels = ['A', 'Aku', 'Apa', 'B', 'Bagaimana', 'Baik', 'Bapak', 'Berapa',
    'Besok', 'C', 'D', 'Dia', 'Dimana', 'E', 'F', 'G', 'H', 'Halo',
    'Hari ini', 'I', 'Ibu', 'J', 'Jawab', 'K', 'Kalian', 'Kamu',
    'Kantor', 'Kapan', 'Kemana', 'Kemarin', 'Kerja', 'L', 'Lelah',
    'Lusa', 'M', 'Maaf', 'Makan', 'Malam', 'Mengapa', 'N', 'Nanti',
    'O', 'P', 'Pagi', 'Q', 'R', 'S', 'Sabar', 'Sakit', 'Sama - sama',
    'Sedih', 'Sekarang', 'Senang', 'Siang', 'Siapa', 'Sore', 'T',
    'Terima kasih', 'Tidur', 'Tolong', 'U', 'V', 'W', 'X', 'Y', 'Z']

# Fungsi untuk menghasilkan daftar kata dari file CSV dengan filter panjang kata
def generate_word_spelling(file_path, del_thresh):
    df = pd.read_csv(file_path, header=None)
    filtered_df = df[df[0].str.len() >= del_thresh]
    words = filtered_df[0].tolist()
    return words

# Fungsi Koreksi Ejaan
def correct_spelling(word, word_list):
    # Prioritas 1: Panjang karakter yang sama
    same_length_words = [w for w in word_list if len(w) == len(word)]
    close_matches = get_close_matches(word, same_length_words, n=1, cutoff=0.8)
    
    if close_matches:
        return close_matches[0]
    
    # Prioritas 2: Ganti karakter dari belakang ke depan
    for i in range(len(word)-1, -1, -1):
        for char in 'abcdefghijklmnopqrstuvwxyz':
            if char != word[i]:  # Hanya mengganti jika karakter berbeda
                possible_word = word[:i] + char + word[i+1:]
                if possible_word in word_list:
                    return possible_word
    
    # Prioritas 3: Tambahkan karakter hanya di akhir jika tidak ada kecocokan
    for char in 'abcdefghijklmnopqrstuvwxyz':
        possible_word = word + char
        if possible_word in word_list:
            return possible_word
    
    return None

# Contoh penggunaan
file_path = './10k-indonesia-common-words.csv'  # Sesuaikan dengan path file Anda
# file_path = './kbbi.csv'  # Sesuaikan dengan path file Anda
del_thresh = 3  # Panjang minimal kata yang diizinkan
list_word = generate_word_spelling(file_path, del_thresh)

# Coba koreksi ejaan
misspelled_word = 'bagaimans'
corrected_word = correct_spelling(misspelled_word, list_word)
print(f"Koreksi untuk '{misspelled_word}': {corrected_word}")


Koreksi untuk 'bagaimans': bagaimana


In [9]:
# Contoh penggunaan
input_word = 'nanti'
closest = correct_spelling(input_word, list_word)
print(f"Kata '{input_word}' paling mendekati label: '{closest}'")

Kata 'nanti' paling mendekati label: 'nanti'


In [10]:
"9_nanti_1".split('_')[1]

'nanti'

In [11]:
'lusa' in list_word

True

In [12]:
import numpy as np

log_entries = []
true = 0
false = 0

for i in range(len(samples)):
    print(f"predicted {samples_path[i]}")
    prediction = model_load.predict(samples[i])
    prediction = np.argmax(prediction, axis=1)
    prediction = class_labels[prediction[0]]
    
    label = samples_path[i].split('_')
    label = label[1]
    
    label = correct_spelling(label, list_word)
    
    status = None
    if label != None:
        if label == prediction:
            status = 'True'
            true += 1
        else:
            status = 'False'
            false += 1
    # else
    # Append log entry to the list
    log_entries.append({
        'samples_path': samples_path[i],
        'status': status,
        'true_value': label,
        'predicted_value': prediction
    })
    print(f'==> {status} | T:{label}|P:{prediction}')

print(f"Done Predict {true+false} data with {true} True and {false} False")            


predicted ./datasets/Testing-Image/11_N_92.jpg
1/1 [==============================] - 0s 104ms/step
==> None | T:None|P:W
predicted ./datasets/Testing-Image/11_q_.jpg
1/1 [==============================] - 0s 15ms/step
==> None | T:None|P:P
predicted ./datasets/Testing-Image/1_ibu_.jpg
1/1 [==============================] - 0s 20ms/step
==> False | T:ibu|P:Dimana
predicted ./datasets/Testing-Image/87_kerja_1.jpg
1/1 [==============================] - 0s 15ms/step
==> False | T:kerja|P:W
predicted ./datasets/Testing-Image/8_kerja_1.jpg
1/1 [==============================] - 0s 16ms/step
==> False | T:kerja|P:N
predicted ./datasets/Testing-Image/8_malam_1.jpg
1/1 [==============================] - 0s 24ms/step
==> False | T:malam|P:B
predicted ./datasets/Testing-Image/8_malam_2.jpg
1/1 [==============================] - 0s 15ms/step
==> False | T:malam|P:B
predicted ./datasets/Testing-Image/8_siang_2.jpg
1/1 [==============================] - 0s 15ms/step
==> False | T:siang|P:W
predicte

In [13]:
df_logs = pd.DataFrame(log_entries)
df_logs['predicted_value'] = df_logs['predicted_value'].str.lower()
wrong_df = df_logs[df_logs['true_value'] != df_logs['predicted_value']]
print(wrong_df.shape)
wrong_df.head(100)

(45, 4)


,samples_path,status,true_value,predicted_value
0,./datasets/Testing-Image/11_N_92.jpg,None,None,w
1,./datasets/Testing-Image/11_q_.jpg,None,None,p
2,./datasets/Testing-Image/1_ibu_.jpg,False,ibu,dimana
3,./datasets/Testing-Image/87_kerja_1.jpg,False,kerja,w
4,./datasets/Testing-Image/8_kerja_1.jpg,False,kerja,n
5,./datasets/Testing-Image/8_malam_1.jpg,False,malam,b
6,./datasets/Testing-Image/8_malam_2.jpg,False,malam,b
7,./datasets/Testing-Image/8_siang_2.jpg,False,siang,w
8,./datasets/Testing-Image/91_jawab_1.jpg,False,jawab,dimana
9,./datasets/Testing-Image/91_kantor_1.jpg,False,kantor,w
